In [2]:
import re
from pprint import pp
DEBUG = False
sample = "x=20..30, y=-10..-5"
not_sample = "x=139..187, y=-148..-89"

get_box = re.compile(r'x=(-?\d+)\.\.(-?\d+), y=(-?\d+)\.\.(-?\d+)').match



class Target:
    def __init__(self, raw):
        data = get_box(raw).groups()
        x1,x2,y1,y2 = [int(d) for d in data]
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
        self.within = lambda x, y: x1 <= x <= x2 and y1 <= y <= y2
        min_y = min((y1,y2))
        self.beyond = lambda y:  y < min_y


    def crunch_ugly(self,xv,yv):
        x = 0
        y = 0
        max_y = y
        c = 0
        while True:
            c+=1
            f = y < max_y
            y == 0 and DEBUG and print (f'{x=} {y=} {c=} {xv=} {yv=} {f=} {max_y=}')
            x+=xv
            y+=yv
            max_y = max((y, max_y))
            yv-=1
            if xv:
                xv-=1
            if self.within(x,y):
                DEBUG and print (f'{x=} {y=} are within: {self.x1=} {self.x2=} {self.y1=} {self.y2=}')
                return max_y
            if f and self.beyond(y):
                DEBUG and print (f'{y=} is beyond: > {self.x2=} or < {self.y2=}')
                break

    
    def work_range(self,xmin,xmax,ymin,ymax):
        DEBUG and print(list(range(xmin,xmax)))
        DEBUG and print(list(range(ymin,ymax)))
        results = {}
        for x in range(xmin,xmax):
            for y in range(ymin,ymax):
                DEBUG and print(f"Working: {x=} {y=}")
                result = self.crunch_ugly(x,y)
                if result is not None:
                    results[(x,y)] = result
                DEBUG and result and print(f"{x=} {y=} {result=}\n")
        return results


DEBUG = True
t = Target(sample)
result = t.work_range(3,9,9,10)
if result:
    print(max(result.values()))



[3, 4, 5, 6, 7, 8]
[9]
Working: x=3 y=9
x=0 y=0 c=1 xv=3 yv=9 f=False max_y=0
x=6 y=0 c=20 xv=0 yv=-10 f=True max_y=45
y=-21 is beyond: > self.x2=30 or < self.y2=-5
Working: x=4 y=9
x=0 y=0 c=1 xv=4 yv=9 f=False max_y=0
x=10 y=0 c=20 xv=0 yv=-10 f=True max_y=45
y=-21 is beyond: > self.x2=30 or < self.y2=-5
Working: x=5 y=9
x=0 y=0 c=1 xv=5 yv=9 f=False max_y=0
x=15 y=0 c=20 xv=0 yv=-10 f=True max_y=45
y=-21 is beyond: > self.x2=30 or < self.y2=-5
Working: x=6 y=9
x=0 y=0 c=1 xv=6 yv=9 f=False max_y=0
x=21 y=0 c=20 xv=0 yv=-10 f=True max_y=45
x=21 y=-10 are within: self.x1=20 self.x2=30 self.y1=-10 self.y2=-5
x=6 y=9 result=45

Working: x=7 y=9
x=0 y=0 c=1 xv=7 yv=9 f=False max_y=0
x=28 y=0 c=20 xv=0 yv=-10 f=True max_y=45
x=28 y=-10 are within: self.x1=20 self.x2=30 self.y1=-10 self.y2=-5
x=7 y=9 result=45

Working: x=8 y=9
x=0 y=0 c=1 xv=8 yv=9 f=False max_y=0
x=36 y=0 c=20 xv=0 yv=-10 f=True max_y=45
y=-21 is beyond: > self.x2=30 or < self.y2=-5
45


In [5]:

import numpy as np
DEBUG = False
t = Target(not_sample)
result = t.work_range(1,188,-200,200)



In [10]:

def result_range(result):
    ys = [y for x,y in result]
    xs = [x for x,y in result]
    xmin = min(xs)
    xmax = max(xs)
    ymin = min(ys)
    ymax = max(ys)
    xoffset = 0 - xmin
    yoffset = 0 - ymin
    print(f'{xmin=} {xmax=} {ymin=} {ymax=} {xoffset=} {yoffset=}')
    shape = (ymax-ymin+1,xmax-xmin+1)
    print(shape)
    array = np.zeros(shape)
    for x,y in result:
        array[y+yoffset,x+xoffset] = 1
    for row in np.flip(array,axis=0):
        print (''.join('X' if e else ' ' for e in row))


print(len(result))
result_range(result)

print(max(result.values()))



4716
xmin=17 xmax=187 ymin=-148 ymax=147 xoffset=-17 yoffset=148
(296, 171)
XX                                                                                                                                                                         
XX                                                                                                                                                                         
XX                                                                                                                                                                         
XX                                                                                                                                                                         
XX                                                                                                                                                                         
XX                                                              

In [11]:
expected1 = """23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
8,-2    27,-8   30,-5   24,-7"""

expected2 = re.findall(r'(-?\d+),(-?\d+)', expected1)

expected = []
for x,y in expected2:
    expected.append((int(x), int(y)))

print(expected)


[(23, -10), (25, -9), (27, -5), (29, -6), (22, -6), (21, -7), (9, 0), (27, -7), (24, -5), (25, -7), (26, -6), (25, -5), (6, 8), (11, -2), (20, -5), (29, -10), (6, 3), (28, -7), (8, 0), (30, -6), (29, -8), (20, -10), (6, 7), (6, 4), (6, 1), (14, -4), (21, -6), (26, -10), (7, -1), (7, 7), (8, -1), (21, -9), (6, 2), (20, -7), (30, -10), (14, -3), (20, -8), (13, -2), (7, 3), (28, -8), (29, -9), (15, -3), (22, -5), (26, -8), (25, -8), (25, -6), (15, -4), (9, -2), (15, -2), (12, -2), (28, -9), (12, -3), (24, -6), (23, -7), (25, -10), (7, 8), (11, -3), (26, -7), (7, 1), (23, -9), (6, 0), (22, -10), (27, -6), (8, 1), (22, -8), (13, -4), (7, 6), (28, -6), (11, -4), (12, -4), (26, -9), (7, 4), (24, -10), (23, -8), (30, -8), (7, 0), (9, -1), (10, -1), (26, -5), (22, -9), (6, 5), (7, 5), (23, -6), (28, -10), (10, -2), (11, -1), (20, -9), (14, -2), (29, -7), (13, -3), (23, -5), (24, -8), (27, -9), (30, -7), (28, -5), (21, -10), (7, 9), (6, 6), (21, -5), (27, -10), (7, 2), (30, -9), (21, -8), (22, -

In [12]:
print (sample)

x=20..30, y=-10..-5


In [16]:
DEBUG = False
t = Target(sample)
result = t.work_range(1,31,-12,100)
if result:
    print(max(result.values()))
print(len(result))
from collections import Counter

counts = Counter(expected + list(result))

pp({k for k,v in counts.items() if v == 1})

result_range(result)

45
112
set()
xmin=6 xmax=30 ymin=-10 ymax=9 xoffset=-6 yoffset=10
(20, 25)
XX                       
XX                       
XX                       
XX                       
XX                       
XX                       
XX                       
XX                       
XXX                      
XXXX                     
 XXXXX                   
  XXXXXXXX               
     XXXXX               
     XXXXX               
              XXXXXXXXXXX
              XXXXXXXXXXX
              XXXXXXXXXXX
              XXXXXXXXXXX
              XXXXXXXXXXX
              XXXXXXXXXXX


In [14]:
DEBUG = True
t = Target(sample)
result = t.work_range(7,8,-1,0)
if result:
    print(max(result.values()))

[7]
[-1]
Working: x=7 y=-1
x=0 y=0 c=1 xv=7 yv=-1 f=False max_y=0
x=22 y=-10 are within: self.x1=20 self.x2=30 self.y1=-10 self.y2=-5
0
